In [42]:
import pandas as pd
import numpy as np

In [142]:
df=pd.read_csv("spam.csv",encoding='latin1')

In [145]:
df=df.drop_duplicates(keep='first')
df['v1'].isnull().sum()

0

In [144]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [48]:
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [148]:
df.rename(columns={'v1':'target','v2':'sms_text'},inplace=True)
df['target'].isnull().sum()

0

In [149]:
df['target']=df['target'].map({'ham':1,'spam':0})
#df['target
print(df['target'].isnull().sum())  # number of NaNs


0


In [111]:
#data preprocessing
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [112]:
ps=PorterStemmer()
stop_words1=set(stopwords.words('english'))
def clean_data(text):
    text=text.lower()
    tokens=nltk.word_tokenize(text)
    tokens=[token for token in tokens if token.isalnum()]
    tokens=[token for token in tokens if token not in stop_words1]
    filtered_words=[ps.stem(token) for token in tokens]
    return " ".join(filtered_words)

In [114]:
df['sms_text']=df['sms_text'].apply(clean_data)

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
tf_matrix=tf.fit_transform(df['sms_text'])

In [126]:
df['target']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
5567   NaN
5568   NaN
5569   NaN
5570   NaN
5571   NaN
Name: target, Length: 5169, dtype: float64

In [170]:
#train test split
from sklearn.model_selection import train_test_split
y=df['target'].values

In [151]:
x_train, x_test,y_train,y_test=train_test_split(tf_matrix,y,test_size=0.2,random_state=50,stratify=y)


In [171]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report,precision_score
model=MultinomialNB()  # creating model class
model.fit(x_train,y_train)  # training the model by providing sms_text and target
y_pred=model.predict(x_test)  # predicting the Y value by passing test data 
print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test,y_pred))

Accuracy: 0.965183752417795
[[ 95  36]
 [  0 903]]
              precision    recall  f1-score   support

           0       1.00      0.73      0.84       131
           1       0.96      1.00      0.98       903

    accuracy                           0.97      1034
   macro avg       0.98      0.86      0.91      1034
weighted avg       0.97      0.97      0.96      1034

0.9616613418530351


In [167]:
test_msg = ["Congrats! 1 year special cinema pass for 2 is yours. call 09061209465 now! C Suprman V, Matrix3, StarWars3, etc all 4 FREE! bx420-ip4-5we. 150pm. Dont miss out! "]
test_vec = tf.transform(test_msg)
print("Spam" if model.predict(test_vec)[0] == 0 else "Ham")

Spam
